In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"




def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=400, output_dim=5000))
    model.add(Activation('tanh'))
    model.add(Dense(512*10*10))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((10, 10, 512), input_shape=(512*10*10,)))
    model.add(UpSampling2D(size=(2, 2)))
    

    model.add(Conv2D(256, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    
    model.add(Conv2D(128, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))    
    
    model.add(Conv2D(3, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model


def discriminator_model():
    model = Sequential()
    model.add(
            Conv2D(32, (5, 5),
            padding='same',
            input_shape=(80, 80, 3))
            )
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model


def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model


def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1],3),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1],:] = \
            img[:, :, :]
    return image


def train(BATCH_SIZE):
    #(X_train, y_train), (X_test, y_test) = mnist.load_data()
    #X_train = (X_train.astype(np.float32) - 127.5)/127.5
    #X_train = X_train[:, :, :, None]
    #X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    d = discriminator_model()
    g = generator_model()
        
    #d.load_weights('discriminator')
    #g.load_weights('generator')

    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.001, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.001, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    for epoch in range(400):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 400))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            if index % 20 == 0:
                image = combine_images(generated_images)
                image = image*255.
                Image.fromarray(image.astype(np.uint8)).save(
                    str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 400))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)

            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)


def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 400))
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 400))
        generated_images = g.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*255.
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args

Using TensorFlow backend.


In [4]:
train(BATCH_SIZE=2)

/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=5000, input_dim=400)`


Epoch is 0
Number of batches 11550
batch 0 d_loss : 1.076535
batch 0 g_loss : 0.252729
batch 1 d_loss : 0.567727
batch 1 g_loss : 0.216877
batch 2 d_loss : 0.396344
batch 2 g_loss : 0.247206
batch 3 d_loss : 0.418348
batch 3 g_loss : 0.353331
batch 4 d_loss : 0.435497
batch 4 g_loss : 0.215798
batch 5 d_loss : 0.475719
batch 5 g_loss : 0.240455
batch 6 d_loss : 0.518008
batch 6 g_loss : 0.443210
batch 7 d_loss : 0.594903
batch 7 g_loss : 0.490328
batch 8 d_loss : 0.699506
batch 8 g_loss : 0.255457
batch 9 d_loss : 0.725856
batch 9 g_loss : 0.289353
batch 10 d_loss : 0.673085
batch 10 g_loss : 0.468621
batch 11 d_loss : 0.588865
batch 11 g_loss : 0.647842
batch 12 d_loss : 0.486397
batch 12 g_loss : 0.953853
batch 13 d_loss : 0.664391
batch 13 g_loss : 1.168144
batch 14 d_loss : 0.516666
batch 14 g_loss : 1.212365
batch 15 d_loss : 0.793064
batch 15 g_loss : 1.079498
batch 16 d_loss : 0.695900
batch 16 g_loss : 0.926258
batch 17 d_loss : 0.655172
batch 17 g_loss : 1.086243
batch 18 d_lo

batch 150 d_loss : 0.181926
batch 150 g_loss : 4.623781
batch 151 d_loss : 0.654156
batch 151 g_loss : 4.492440
batch 152 d_loss : 0.250676
batch 152 g_loss : 2.812252
batch 153 d_loss : 0.099427
batch 153 g_loss : 2.911113
batch 154 d_loss : 1.222983
batch 154 g_loss : 2.726950
batch 155 d_loss : 1.068208
batch 155 g_loss : 5.750004
batch 156 d_loss : 0.680485
batch 156 g_loss : 2.327156
batch 157 d_loss : 0.364644
batch 157 g_loss : 3.316209
batch 158 d_loss : 0.216344
batch 158 g_loss : 0.975131
batch 159 d_loss : 0.040093
batch 159 g_loss : 2.408675
batch 160 d_loss : 0.754540
batch 160 g_loss : 2.804924
batch 161 d_loss : 0.176001
batch 161 g_loss : 3.365695
batch 162 d_loss : 0.192658
batch 162 g_loss : 3.437530
batch 163 d_loss : 0.146196
batch 163 g_loss : 1.556417
batch 164 d_loss : 0.920118
batch 164 g_loss : 1.435178
batch 165 d_loss : 0.753290
batch 165 g_loss : 3.338095
batch 166 d_loss : 1.272836
batch 166 g_loss : 4.796068
batch 167 d_loss : 0.306099
batch 167 g_loss : 3

batch 297 g_loss : 3.142452
batch 298 d_loss : 0.740360
batch 298 g_loss : 5.851087
batch 299 d_loss : 0.688681
batch 299 g_loss : 3.364676
batch 300 d_loss : 0.208467
batch 300 g_loss : 3.396394
batch 301 d_loss : 0.098747
batch 301 g_loss : 3.975781
batch 302 d_loss : 1.562170
batch 302 g_loss : 0.915245
batch 303 d_loss : 0.264914
batch 303 g_loss : 5.112307
batch 304 d_loss : 0.315435
batch 304 g_loss : 3.658021
batch 305 d_loss : 0.343965
batch 305 g_loss : 4.047324
batch 306 d_loss : 0.431345
batch 306 g_loss : 3.189137
batch 307 d_loss : 0.514145
batch 307 g_loss : 2.544212
batch 308 d_loss : 0.357526
batch 308 g_loss : 3.727373
batch 309 d_loss : 0.539972
batch 309 g_loss : 2.138735
batch 310 d_loss : 1.060680
batch 310 g_loss : 3.322462
batch 311 d_loss : 0.175666
batch 311 g_loss : 3.861755
batch 312 d_loss : 0.391060
batch 312 g_loss : 2.499542
batch 313 d_loss : 0.171947
batch 313 g_loss : 5.624956
batch 314 d_loss : 0.232112
batch 314 g_loss : 2.623066
batch 315 d_loss : 0

batch 444 g_loss : 5.179347
batch 445 d_loss : 0.028520
batch 445 g_loss : 1.629603
batch 446 d_loss : 2.107128
batch 446 g_loss : 1.466602
batch 447 d_loss : 0.025485
batch 447 g_loss : 0.105232
batch 448 d_loss : 1.061503
batch 448 g_loss : 2.164683
batch 449 d_loss : 0.431209
batch 449 g_loss : 2.198454
batch 450 d_loss : 0.515676
batch 450 g_loss : 1.395064
batch 451 d_loss : 0.441718
batch 451 g_loss : 1.766452
batch 452 d_loss : 0.243147
batch 452 g_loss : 2.854257
batch 453 d_loss : 0.191246
batch 453 g_loss : 0.913340
batch 454 d_loss : 0.122634
batch 454 g_loss : 1.806508
batch 455 d_loss : 0.042770
batch 455 g_loss : 0.455400
batch 456 d_loss : 0.047776
batch 456 g_loss : 3.600810
batch 457 d_loss : 1.133877
batch 457 g_loss : 0.164974
batch 458 d_loss : 0.074462
batch 458 g_loss : 0.149912
batch 459 d_loss : 0.964124
batch 459 g_loss : 0.341586
batch 460 d_loss : 0.695485
batch 460 g_loss : 0.781067
batch 461 d_loss : 0.501826
batch 461 g_loss : 2.067905
batch 462 d_loss : 0

...

batch 1456 d_loss : 0.021248
batch 1456 g_loss : 4.797259
batch 1457 d_loss : 0.022273
batch 1457 g_loss : 4.614153
batch 1458 d_loss : 0.028080
batch 1458 g_loss : 4.630891
batch 1459 d_loss : 0.026443
batch 1459 g_loss : 4.567641
batch 1460 d_loss : 0.012058
batch 1460 g_loss : 4.405703
batch 1461 d_loss : 0.053131
batch 1461 g_loss : 2.192083
batch 1462 d_loss : 0.016370
batch 1462 g_loss : 3.253097
batch 1463 d_loss : 0.059694
batch 1463 g_loss : 5.225205
batch 1464 d_loss : 0.010190
batch 1464 g_loss : 4.949121
batch 1465 d_loss : 0.010923
batch 1465 g_loss : 4.912542
batch 1466 d_loss : 0.890530
batch 1466 g_loss : 2.009681
batch 1467 d_loss : 1.411775
batch 1467 g_loss : 0.827459
batch 1468 d_loss : 2.239925
batch 1468 g_loss : 2.094645
batch 1469 d_loss : 0.037184
batch 1469 g_loss : 2.148770
batch 1470 d_loss : 1.311019
batch 1470 g_loss : 8.065788
batch 1471 d_loss : 0.294429
batch 1471 g_loss : 8.062521
batch 1472 d_loss : 1.696469
batch 1472 g_loss : 8.062904
batch 1473 d_l

batch 1598 g_loss : 3.500738
batch 1599 d_loss : 0.132630
batch 1599 g_loss : 3.893800
batch 1600 d_loss : 0.116884
batch 1600 g_loss : 4.049856
batch 1601 d_loss : 0.090707
batch 1601 g_loss : 4.145327
batch 1602 d_loss : 0.130518
batch 1602 g_loss : 2.576423
batch 1603 d_loss : 0.080177
batch 1603 g_loss : 3.270280
batch 1604 d_loss : 0.085768
batch 1604 g_loss : 4.272768
batch 1605 d_loss : 0.077267
batch 1605 g_loss : 1.609197
batch 1606 d_loss : 0.127844
batch 1606 g_loss : 2.844315
batch 1607 d_loss : 0.188959
batch 1607 g_loss : 3.099469
batch 1608 d_loss : 0.097275
batch 1608 g_loss : 3.669873
batch 1609 d_loss : 0.098110
batch 1609 g_loss : 3.869254
batch 1610 d_loss : 0.035369
batch 1610 g_loss : 2.462975
batch 1611 d_loss : 0.157095
batch 1611 g_loss : 8.539176
batch 1612 d_loss : 0.138529
batch 1612 g_loss : 9.151878
batch 1613 d_loss : 0.050199
batch 1613 g_loss : 9.206482
batch 1614 d_loss : 0.060800
batch 1614 g_loss : 9.078053
batch 1615 d_loss : 0.061070
batch 1615 g_l

batch 1739 g_loss : 4.565146
batch 1740 d_loss : 0.476688
batch 1740 g_loss : 8.187326
batch 1741 d_loss : 0.493668
batch 1741 g_loss : 8.191651
batch 1742 d_loss : 0.853177
batch 1742 g_loss : 8.218719
batch 1743 d_loss : 0.474292
batch 1743 g_loss : 8.243834
batch 1744 d_loss : 0.461091
batch 1744 g_loss : 8.275143
batch 1745 d_loss : 0.203351
batch 1745 g_loss : 8.267636
batch 1746 d_loss : 0.579398
batch 1746 g_loss : 8.292911
batch 1747 d_loss : 0.447722
batch 1747 g_loss : 6.690326
batch 1748 d_loss : 0.457648
batch 1748 g_loss : 8.315898
batch 1749 d_loss : 0.673539
batch 1749 g_loss : 8.469649
batch 1750 d_loss : 0.457796
batch 1750 g_loss : 8.359710
batch 1751 d_loss : 0.448504
batch 1751 g_loss : 8.432568
batch 1752 d_loss : 0.449891
batch 1752 g_loss : 8.342617
batch 1753 d_loss : 0.553092
batch 1753 g_loss : 7.151994
batch 1754 d_loss : 0.379997
batch 1754 g_loss : 4.672580
batch 1755 d_loss : 0.654741
batch 1755 g_loss : 8.363029
batch 1756 d_loss : 0.308223
batch 1756 g_l

batch 1881 g_loss : 4.263278
batch 1882 d_loss : 0.187370
batch 1882 g_loss : 8.313725
batch 1883 d_loss : 0.398533
batch 1883 g_loss : 8.346860
batch 1884 d_loss : 0.454143
batch 1884 g_loss : 1.987202
batch 1885 d_loss : 0.340225
batch 1885 g_loss : 8.548485
batch 1886 d_loss : 0.245021
batch 1886 g_loss : 8.534083
batch 1887 d_loss : 0.319753
batch 1887 g_loss : 3.152498
batch 1888 d_loss : 0.312728
batch 1888 g_loss : 5.204484
batch 1889 d_loss : 0.222556
batch 1889 g_loss : 8.529860
batch 1890 d_loss : 0.392089
batch 1890 g_loss : 8.726815
batch 1891 d_loss : 0.254960
batch 1891 g_loss : 4.930072
batch 1892 d_loss : 0.224405
batch 1892 g_loss : 2.270386
batch 1893 d_loss : 0.225281
batch 1893 g_loss : 6.202731
batch 1894 d_loss : 0.185239
batch 1894 g_loss : 1.229337
batch 1895 d_loss : 0.324845
batch 1895 g_loss : 5.438487
batch 1896 d_loss : 0.281131
batch 1896 g_loss : 2.541394
batch 1897 d_loss : 0.245051
batch 1897 g_loss : 2.981272
batch 1898 d_loss : 0.169626
batch 1898 g_l

KeyboardInterrupt: 

In [5]:
generate(BATCH_SIZE=80, nice=False)

/home/afattahov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=5000, input_dim=400)`


80/80 [==============================] - 0s     


In [8]:
X_train.shape

(60000, 28, 28)

In [2]:
import random
import cv2
random.randint(0,2)
import os
files = os.listdir('data/zir')

In [11]:
# Make gan_data
idx = 0
for t in range(30):
    for i in files:
        img = cv2.imread('data/zir/'+i)
        for j in range(10):
            x = random.randint(200,img.shape[0]-400)
            y = random.randint(200,img.shape[1]-400)
            tmp_img = img[x:x+80,y:y+80,:]
            cv2.imwrite('gan_data224/{}.jpg'.format(idx),tmp_img)
            idx += 1

In [3]:
import os
files = os.listdir('gan_data224')

import cv2

X_train = []
for i in files:
    tmp = cv2.imread('gan_data224/'+i)
    X_train += [tmp]
X_train = np.array(X_train)

X_train = X_train.astype(np.float32) / 255.

In [13]:
X_train.shape

(23100, 80, 80, 3)